In [55]:
import requests
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import datetime

In [40]:
r = requests.get("https://data.binance.com/api/v3/depth",
                 params=dict(symbol="OSMOUSDT"))
results = r.json()

In [41]:
# create a dictionary of dataframe
frames = {side: pd.DataFrame(data=results[side], columns=["price", "quantity"],
                             dtype=float)
          for side in ["bids", "asks"]}

In [42]:
print(frames)

{'bids':     price  quantity
0   0.855    360.55
1   0.854    921.78
2   0.853  21365.57
3   0.852   9942.97
4   0.851    469.37
..    ...       ...
95  0.727     27.51
96  0.725     30.79
97  0.724    414.36
98  0.723     69.15
99  0.722     46.60

[100 rows x 2 columns], 'asks':     price  quantity
0   0.856     42.64
1   0.857   1245.70
2   0.858   1050.43
3   0.859   8852.43
4   0.860   1219.11
..    ...       ...
95  0.987     12.38
96  0.988     11.41
97  0.989    559.86
98  0.990   7252.34
99  0.992    769.01

[100 rows x 2 columns]}


In [47]:
frames_list = [frames[side].assign(side=side) for side in frames]
data = pd.concat(frames_list, axis="index", 
                 ignore_index=True, sort=True)
print(data)

     price  quantity  side
0    0.855    360.55  bids
1    0.854    921.78  bids
2    0.853  21365.57  bids
3    0.852   9942.97  bids
4    0.851    469.37  bids
..     ...       ...   ...
195  0.987     12.38  asks
196  0.988     11.41  asks
197  0.989    559.86  asks
198  0.990   7252.34  asks
199  0.992    769.01  asks

[200 rows x 3 columns]


In [45]:
frames["bids"].price.nlargest(5)

0    0.855
1    0.854
2    0.853
3    0.852
4    0.851
Name: price, dtype: float64

In [46]:
frames["asks"].price.nsmallest(5)

0    0.856
1    0.857
2    0.858
3    0.859
4    0.860
Name: price, dtype: float64

In [58]:
book_list = []

In [59]:
# keep getting orderbook from binance, 
def get_binance_book():
    r = requests.get("https://data.binance.com/api/v3/depth",
                 params=dict(symbol="OSMOUSDT"))
    results = r.json()
    # a dictionary of side: price, quantity orderbook
    frame = {side: pd.DataFrame(data=results[side], columns=["price", "quantity"],
                             dtype=float)
          for side in ["bids", "asks"]}
    book_list += frame
    return book_list

In [60]:
def get_block_time():
    block_request = requests.get('https://osmosis-lcd.quickapi.com/cosmos/base/tendermint/v1beta1/blocks/latest').json()
    blockNumber = int(block_request['block']['header']['height'])
    blockTime = block_request['block']['header']['time']
    blockTime = blockTime.replace('Z', '')[0:19]
    blockTime = datetime.datetime.strptime(blockTime,"%Y-%m-%dT%H:%M:%S")
    return blockTime, blockNumber

In [61]:
saved_block_time = get_block_time()[0]
while True:
    book_list = get_binance_book()
    block_time = get_block_time()[0]
    if block_time > saved_block_time:
        print(book_list[-1])
        saved_block_time = block_time
        break

UnboundLocalError: local variable 'book_list' referenced before assignment